## Gathering Dependencies

_Importing Required Libraries_

In [15]:
pip install hampel

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\Anurag Dutta\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [16]:
from keras.models import Sequential
from keras.layers import Dropout
from keras.layers import Dense
from keras.layers import LSTM
import keras
import tensorflow as tf
from hampel import hampel
import numpy as np
from numpy import array
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt
from matplotlib import pyplot
from numpy import array
from tensorflow.keras.callbacks import EarlyStopping

## Loading Datasets

_SAN JUAN_

In [17]:
data = pd.read_csv("datasets/sanjuan.csv")
training_set = data.iloc[:, 3]
print(training_set)

0        4
1        5
2        4
3        3
4        6
        ..
1191    56
1192    46
1193    52
1194    34
1195    25
Name: Cases, Length: 1196, dtype: int64


## Preprocessing the Data

In [18]:
# Multivariate data preparation
from numpy import array

# split a Multivariate sequence into samples
def split_sequence(sequence, n_steps_in, n_steps_out, features):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps_in
		out_end_ix = end_ix + n_steps_out
		# check if we are beyond the sequence
		if out_end_ix > len(sequence):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix:out_end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

# choose a number of time steps
n_steps_out = 52
n_steps_in = 10
n_features = 1

# split into samples
X, y = split_sequence(training_set, n_steps_in, n_steps_out, n_features)
# summarize the data
for i in range(len(X)):
	print(X[i], y[i])

[ 4  5  4  3  6  2  4  5 10  6] [ 8  2  6 17 23 13 21 28 24 20 40 27 42 33 43 37 57 71 44 56 53 52 47 26
 27 21 21 26 34 37 17 19 25 18 21 17 17 16 16 15 23 16 17 12 17 10 15 19
 21 14 18 13]
[ 5  4  3  6  2  4  5 10  6  8] [ 2  6 17 23 13 21 28 24 20 40 27 42 33 43 37 57 71 44 56 53 52 47 26 27
 21 21 26 34 37 17 19 25 18 21 17 17 16 16 15 23 16 17 12 17 10 15 19 21
 14 18 13 14]
[ 4  3  6  2  4  5 10  6  8  2] [ 6 17 23 13 21 28 24 20 40 27 42 33 43 37 57 71 44 56 53 52 47 26 27 21
 21 26 34 37 17 19 25 18 21 17 17 16 16 15 23 16 17 12 17 10 15 19 21 14
 18 13 14 18]
[ 3  6  2  4  5 10  6  8  2  6] [17 23 13 21 28 24 20 40 27 42 33 43 37 57 71 44 56 53 52 47 26 27 21 21
 26 34 37 17 19 25 18 21 17 17 16 16 15 23 16 17 12 17 10 15 19 21 14 18
 13 14 18 23]
[ 6  2  4  5 10  6  8  2  6 17] [23 13 21 28 24 20 40 27 42 33 43 37 57 71 44 56 53 52 47 26 27 21 21 26
 34 37 17 19 25 18 21 17 17 16 16 15 23 16 17 12 17 10 15 19 21 14 18 13
 14 18 23 25]
[ 2  4  5 10  6  8  2  6 17 23] [13 21 2

## Train and Test Split

In [19]:
train_size = int(len(y) * 0.8)
test_size = len(y) - train_size
dataX = np.array(X)
dataY = np.array(y)
trainX = np.array(X[0:train_size])
trainY = np.array(y[0:train_size])
testX = np.array(X[train_size:len(X)])
testY = np.array(y[train_size:len(y)])
trainX = trainX.reshape((trainX.shape[0], 1, trainX.shape[1]))
testX = testX.reshape((testX.shape[0], 1, testX.shape[1]))
print(trainX.shape, trainY.shape, testX.shape, testY.shape)

(908, 1, 10) (908, 52) (227, 1, 10) (227, 52)


## Model

In [20]:
# split a Multivariate sequence into samples
def split_sequence(sequence, n_steps_in, n_steps_out, features):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps_in
		out_end_ix = end_ix + n_steps_out
		# check if we are beyond the sequence
		if out_end_ix > len(sequence):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix:out_end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

# choose a number of time steps
n_steps_out = 52
n_steps_in = 10
n_features = 1

# split into samples
X, y = split_sequence(training_set, n_steps_in, n_steps_out, n_features)

n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
test_days = 227
trainX, trainY = X[:-test_days], y[:-test_days]
testX, testY = X[-test_days:], y[-test_days:]

custom_early_stopping = EarlyStopping(
    monitor='val_loss', 
    patience=1, 
    min_delta=10, 
    mode='min'
)

model = Sequential()
model.add(LSTM(50, input_shape=(trainX.shape[1], trainX.shape[2])))
model.add(Dense(52))
model.compile(loss='mean_squared_error', optimizer='adam')

history = model.fit(trainX, trainY, epochs=500, batch_size=64, validation_data=(testX, testY), shuffle=False, callbacks=[custom_early_stopping])

Epoch 1/500
15/15 [==============================] - 2s 37ms/step - loss: 3893.6824 - val_loss: 5750.3765
Epoch 2/500
15/15 [==============================] - 0s 8ms/step - loss: 3875.3821 - val_loss: 5720.7246
Epoch 3/500
15/15 [==============================] - 0s 10ms/step - loss: 3849.3865 - val_loss: 5672.4941
Epoch 4/500
15/15 [==============================] - 0s 8ms/step - loss: 3808.1958 - val_loss: 5598.1025
Epoch 5/500
15/15 [==============================] - 0s 8ms/step - loss: 3751.1458 - val_loss: 5501.8438
Epoch 6/500
15/15 [==============================] - 0s 10ms/step - loss: 3686.8538 - val_loss: 5401.1372
Epoch 7/500
15/15 [==============================] - 0s 8ms/step - loss: 3625.5815 - val_loss: 5318.0273
Epoch 8/500
15/15 [==============================] - 0s 8ms/step - loss: 3571.7078 - val_loss: 5240.1655
Epoch 9/500
15/15 [==============================] - 0s 8ms/step - loss: 3522.3132 - val_loss: 5165.8418
Epoch 10/500
15/15 [==============================] 

## Metrics

In [21]:
yhat = model.predict(testX)
print(yhat.shape)
testX = testX.reshape((testX.shape[0], testX.shape[2]*10))
print(testX.shape)
inv_yhat = np.concatenate((testX, yhat), axis=1)
inv_yhat1 = inv_yhat[:, -3:]
inv_yhat = inv_yhat[:, -3]
inv_y = np.concatenate((testX, testY), axis=1)
inv_y1 = inv_y[:, -3:]
inv_y = inv_y[:, -3]
rmse = np.sqrt(mean_squared_error(inv_y, inv_yhat))
mae = mean_absolute_error(inv_y, inv_yhat)
print('Test RMSE: %.3f' % rmse)
print('Test MAE: %.3f' % mae)

8/8 [==============================] - 0s 2ms/step
(227, 52)
(227, 10)
Test RMSE: 72.184
Test MAE: 43.242
